## WP04.01 Coverage

The requirement it to have a certain point within the area of interest (AoI) covered (i.e. observed by the payload) twice a day (FireSat II req. 34.3)

Model works in the following sequence (implemente in FreeFlyer)
- Define and Area of Interest (for our case a subset of Amazonia)
- Define points within the AoI (discretizes the area of interest by the number of "points" resolution set up). For our case we use 40984, which divides the AoI in,  

$$ S_{AoI} = \frac{1615668 [km^2]}{40984} \approx 39.4 [km^2]$$
 
 ![Area of Interest in case study](Figs/Area%20AOI.png)

 Figure 04.01.01: Area of the AoI obtained through the platform Google Earth. Source: the author.

 ![Amount of Points inside the region](Figs/7sats%20coverage.png)

 Figure 04.01.02: Amount of points inside the region based on the results given by the platform. Source: the author.



From literature, swath of 20 km exist, and thus this discretization would be a $20[km]\times 2 [km]$ scene whichis feasible by analogy. The temporal resolution is estimated by counting the number of passes the satellite has over each point and dividing by the number of days. Some areas will be covered more times than others. Thus the AOI temporal resolution is estimated as,

$$ R_{temporal,AoI}\approx \frac {N_{passes,AoI,day}}{N_{points}} $$

The only aditional aspect would be to determine the number of data points in the AoI as a percetange whihc fulfill the required 2 or more passses per day.

In total it returns the frecuency of passing per day. Based on the requirements from FireSat II the amount of passes on the designated area shall be at least twice per day. Thus, an iterative process is executed where the main parameters to trade are the Field of View (FOV) (or in Freeflyer the so called "half angle", half the FOV) and the type of orbit.

## WP04.01 Spatial resolution INPUTS

This code estimates the input parameters for the sizing of the spatial resolution data rate used in the spatial resolution sizing relations in WP04.01.

### Field of View (FOV)

The Field of View (FOV) corresponds to the solid angle through which a detector element (a pixel sensor) is sensitive to electromagnetic radiation at any one time. In that case is called instantaneous field of view or IFOV. A measure of the spatial resolution of a remote sensing imaging system, it is often expressed as dimensions of visible ground area, for some known sensor altitude. For this analysis scenario it is established that the view of the satellite will not be based from a cone but rather a rectangle, using the CAM sensor used by the KITSUNE spacecraft. After that, the swath width can be estimated.

We begin by the spatial resolutions provided previosly in WP04, named $R_{GSD}$. With this information you can estimate the swath width by multiplying it with sizes of $6464 \times 4852 [px]$ based on Kitsune (Azami et al, 2022).Then, based on the literature followed by New SMAD (2011), you can calculate the half angles of the FOV of the sensor pointing nadir with this upcoming equations. It is mentioned that the instrumental focal length $f$ is obtained as well.

$$
 S_W = 2h\tan \alpha_a [m]
$$

$$
\alpha _a = \arctan \left ( \frac{x_a}{f} \right ) [°], 
\alpha _b = \arctan \left ( \frac{x_b}{f} \right ) [°]
$$

![nomenclature](Figs/Field%20of%20View%20Nomenclature%20Diagram.png)

Fig. 04.01.03: Nomenclature of the sensor's filed of view (FOV) and parameters to be calculated. Source: the author.

Where $\alpha _a, \alpha _b$ are the angles corresponding the width and height measured from the center lines respectively, $x_a, x_b$ are the half width and half height of the sensor respectively.

In [1]:
import numpy as np
import pandas as pd
# Spatial resolution is required to be 50 [m] Req 34.1.2

R_GSD= np.array([5, 10, 20, 25, 50]) # [m] 
# Ground Sample Distance 25 [m] is used for 1st iteration considering some image quality considerations in CubeSat telescopes.

h = np.array([2.75e5, 3.5e5, 4.6e5, 4.5e5, 6e5]) #[m] the mission altitudes 

S_w= 6464 * R_GSD # [m] Estimated from equivalent CubeSat for fire monitoring mission. The 6464 value is the amount of pixels in the width of the sensor used by KITSUNE's CAM [Asamis et al, 2022].


#initializing arrays
alpha_a = np.zeros((len(h), len(R_GSD)))
f = np.zeros((len(h), len(R_GSD)))
alpha_b = np.zeros((len(h), len(R_GSD)))


px_size = 3.4736e-6 # [m] estimated from Eq. 1 and data provided by Asif et al 2022.
#it is calculated with the focal length of 300 mm of KITSUNE's CAM, and either the 4.4 [m] in R_GSD at 380 [km] or 4.8 [m] at 420 [km]. Then, Eq. 1 is used to get px_size.

x_a = 6464 *px_size / 2 # [m] size of the half of the sensor's width [Azami et al, 2022]
x_b = 4852 *px_size / (2*4) # [m] size of the half of the sensor's height [Azami et al, 2022].
# Assuming panchromatic sensor and NIR present, the sensor will be split evenly on it height 4 times.

diferencias = {}

for i, r in enumerate(h):
    diff = np.empty((0,4))
    for resolution in range(len(R_GSD)): # the equations shown above to calculate the afore mentioned.
        
        alpha_a = np.degrees(np.arctan(S_w[resolution] / (2*r))) # width half angle
        f = x_a / np.tan(np.radians(alpha_a)) #focal length
        alpha_b =  np.degrees(np.arctan(x_b/f)) #height half angle

        diff = np.vstack((diff, [R_GSD[resolution], np.round(alpha_a, 2), np.round(alpha_b, 2), np.round(f*1000,2)]))
    
    diferencias[f'{i+1}, h= {r/1000} [km]'] = pd.DataFrame(diff, columns=['Ground Sample Distance R_GSD [m]',
                                                              'Half Width Angle alpha_a [°]',
                                                              'Half Height Angle alpha_b [°]',
                                                              'Instrumental Focal Length f [mm]']) #build the tables


# Display
for key, df in diferencias.items():
    print(f"Scenario {key}:")
    display(df)
    print()


Scenario 1, h= 275.0 [km]:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,3.36,0.63,191.05
1,10.0,6.70,1.26,95.52
2,20.0,13.23,2.53,47.76
3,25.0,16.37,3.16,38.21
4,50.0,30.44,6.29,19.10



Scenario 2, h= 350.0 [km]:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,2.64,0.50,243.15
1,10.0,5.28,0.99,121.58
2,20.0,10.46,1.98,60.79
3,25.0,13.00,2.48,48.63
4,50.0,24.78,4.95,24.32



Scenario 3, h= 460.0 [km]:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,2.01,0.38,319.57
1,10.0,4.02,0.76,159.79
2,20.0,8.00,1.51,79.89
3,25.0,9.96,1.89,63.91
4,50.0,19.36,3.77,31.96



Scenario 4, h= 450.0 [km]:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,2.06,0.39,312.62
1,10.0,4.11,0.77,156.31
2,20.0,8.17,1.54,78.16
3,25.0,10.18,1.93,62.52
4,50.0,19.75,3.86,31.26



Scenario 5, h= 600.0 [km]:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,1.54,0.29,416.83
1,10.0,3.08,0.58,208.42
2,20.0,6.15,1.16,104.21
3,25.0,7.67,1.45,83.37
4,50.0,15.07,2.89,41.68


# References

- Larson, W; Wertz, J. Space Mission Analysis and Design. 2016. Ch.8, p.167-168.
- Azami, M. H. bin, Orger, N. C., Schulz, V. H., Oshiro, T., Alarcon, J. R. C., Maskey, A., Nakayama, K., Fukuda, Y., Kojima, K., Yamauchi, T., Masui, H., & Cho, M. (2022). Design and environmental testing of imaging payload for a 6 U CubeSat at low Earth orbit: KITSUNE mission. Frontiers in Space Technologies, 3(November), 1–16. https://doi.org/10.3389/frspt.2022.1000219
- Wertz et al New SMAD 2011
